<a href="https://colab.research.google.com/github/JoaoMKlGui/LearningPytorch/blob/main/IdentifyingNumbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torch import save, load 
import torch.nn as nn
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor 

train = datasets.MNIST(root="data", download=True, train=True, transform=ToTensor()) 

#root fala de onde vamos tirar os dados aparentemente
#o download é pra indicar se vamos ter que fazer o download ou se já está na máquina o dataset
#o arg train é pra separar o dataset em um dataset feito para treino 
#como é imagem, utilizamos o transform para falar como queremos manipular/transformar o input. Nesse caso, mudando para um tensor

dataset = DataLoader(train, batch_size=32)

#utilizando a estratégia de dividir em batches de 32 samples cada e passando train(nossa variavel que guarda as imagens) para o DataLoader fazer as divisões


class ImageClassifier(nn.Module):  #declarando a classe ImageClassifier como sendo uma subclasse de nn.Module

  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Conv2d(1, 32, (3,3)), #como vamos classificar e trabalhar com imagens, utilizar redes convolocionais parece ser a melhor opcao. O tamanho do input é 1 pois a imagem é em preto e branco, logo, só possui 1 camada (seria 3 se fosse rgb eu acho)
        nn.ReLU(), #primeira funcao de ativacao é a relu
        nn.Conv2d(32, 64, (3,3)), 
        nn.ReLU(),
        nn.Conv2d(64, 128, (3,3)), #a partir daqui eu adicionei as camadas para ir aprendendo algumas coisas. nada foi pego do vídeo
        nn.ReLU(),
        nn.Conv2d(128,128, (3,3)),
        nn.ReLU(),
        nn.Conv2d(128,128, (3,3)),
        nn.Flatten(), #transformando a imagem em um tensor de dimensão única
        nn.Linear(128 * (28-10) * (28-10), 10) #última camada. São 256 entradas e 256 saídas. A cada camada da RN, a gente tá tirando 2 pixels (não sei o motivo já que o cara do vídeo não falou). Então devemos subtrair do shape das imagens 2*4 = 8
                 #o output deve ser o número de classes (no nosso caso, vai de 0 a 9). logo, 10 é o tamanho
    )

  def forward(self, x):
    return self.model(x)


#criar as instancias da rede neural

clf = ImageClassifier().to('cuda')     #pra acelerar o processo vamos mandar pra GPU as coisas (não entendi muito bem essa parte de cuda, mas sei que é pra acelerar o processo porque é mais rápido)
optimizer = SGD(clf.parameters(), lr = 0.01)
lossFunction = nn.CrossEntropyLoss()

# criando a parte de treinamento

for epoch in range(100): #treinando para 10 epochs
  for batch in dataset: #vamos treinar uma vez de cada para cada divisão de dados
    X, Y = batch # cada elemento (batch) do dataset (divisão feita pelo dataloader) vem com 2 elementos, o X e o Y
    X, Y = X.to('cuda'), Y.to('cuda') #mandando nossos dados para a gpu
    
    yPredict = clf(X) #criando as previsões

    loss = lossFunction(yPredict, Y) #passando a previsão e o Y esperado para calcular a perda
    
    #aplicando backprop
    optimizer.zero_grad() #se tiver algum gradiente na memoria (eu acho que eles ficam na memoria, n sei), preciso zerar eles
    loss.backward() #calculando o gradiente do backprop
    optimizer.step()

  print("epoch", epoch, ": ", loss.item())

with open('modelo.pt', 'wb') as f:
  save(clf.state_dict(), f)



100%|██████████| 9912422/9912422 [00:00<00:00, 407748923.58it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 33658153.33it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 101540029.31it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 7826840.09it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

epoch 0 :  0.07968434691429138
epoch 1 :  0.055770471692085266
epoch 2 :  0.027678372338414192
epoch 3 :  0.017150210216641426
epoch 4 :  0.012407481670379639
epoch 5 :  0.009691692888736725
epoch 6 :  0.007801945321261883
epoch 7 :  0.006557391956448555
epoch 8 :  0.005838211625814438
epoch 9 :  0.005194349214434624
epoch 10 :  0.004582924768328667
epoch 11 :  0.003987473901361227
epoch 12 :  0.0035695875994861126
epoch 13 :  0.003277648240327835
epoch 14 :  0.003266396000981331
epoch 15 :  0.0034250349272042513
epoch 16 :  0.0037925466895103455
epoch 17 :  0.004389398265630007
epoch 18 :  0.0020485343411564827
epoch 19 :  0.000824176415335387
epoch 20 :  8.197729039238766e-05
epoch 21 :  8.328775584232062e-05
epoch 22 :  7.860360346967354e-05
epoch 23 :  6.613073492189869e-05
epoch 24 :  4.8250079998979345e-05
epoch 25 :  4.299608917790465e-05
epoch 26 :  8.020235327421688e-06
epoch 27 :  4.4017830077791587e-05
ep

In [ ]:
from PIL import Image 
import torch

with open('modelo.pt', 'rb') as f:
  clf.load_state_dict(load(f))

img = Image.open('img_1.jpg')
img.convert('1')
imgTensor = ToTensor()(img).unsqueeze(0).to('cuda')
print(torch.argmax(clf(imgTensor)))

tensor(2, device='cuda:0')


In [ ]:
img2 = Image.open('img_2.jpg')
img2.convert('1')
img2Tensor = ToTensor()(img2).unsqueeze(0).to('cuda')
print(torch.argmax(clf(imtg2Tensor)))

tensor(0, device='cuda:0')
